# HW3 Image Classification
## We strongly recommend that you run with [Kaggle](https://www.kaggle.com/t/86ca241732c04da99aca6490080bae73) for this homework

If you have any questions, please contact the TAs via TA hours, NTU COOL, or email to mlta-2023-spring@googlegroups.com

# Check GPU Type

In [1]:
!nvidia-smi

Fri Mar 17 08:10:41 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    25W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Get Data
Notes: if the links are dead, you can download the data directly from Kaggle and upload it to the workspace, or you can use the Kaggle API to directly download the data into colab.


In [2]:
# Download Link
# Link 1 (Dropbox): https://www.dropbox.com/s/up5q1gthsz3v0dq/food-11.zip?dl=0
# Link 2 (Google Drive): https://drive.google.com/file/d/1tbGNwk1yGoCBdu4Gi_Cia7EJ9OhubYD9/view?usp=share_link
# Link 3: Kaggle Competition.
!pip install --upgrade --no-cache-dir gdown
# (1) dropbox link
#!wget -O food11.zip https://drive.google.com/uc?id=10bL4jEDmM4z9uzGt5KoIq8HEyW6blnLy
# (2) google drive link
!gdown --id '10bL4jEDmM4z9uzGt5KoIq8HEyW6blnLy' --output food11.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=10bL4jEDmM4z9uzGt5KoIq8HEyW6blnLy
To: /content/food11.zip
100% 1.16G/1.16G [00:07<00:00, 158MB/s]


In [3]:
! unzip food11.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: train/9_4311.jpg        
  inflating: train/9_4376.jpg        
  inflating: train/9_7584.jpg        
  inflating: train/9_9259.jpg        
  inflating: train/9_7284.jpg        
  inflating: train/9_1748.jpg        
  inflating: train/9_1630.jpg        
  inflating: train/9_3000.jpg        
  inflating: train/9_3252.jpg        
  inflating: train/9_3880.jpg        
  inflating: train/9_9687.jpg        
  inflating: train/9_7097.jpg        
  inflating: train/9_453.jpg         
  inflating: train/9_1685.jpg        
  inflating: train/9_5144.jpg        
  inflating: train/9_4010.jpg        
  inflating: train/9_8086.jpg        
  inflating: train/9_2881.jpg        
  inflating: train/9_6089.jpg        
  inflating: train/9_9497.jpg        
  inflating: train/9_3328.jpg        
  inflating: train/9_7954.jpg        
  inflating: train/9_4958.jpg        
  inflating: train/9_1319.jpg        
  inflating: train/9_3666.jpg        
  inflating: train/2_1577.jpg

# Import Packages

In [4]:
_exp_name = "0317esm"

In [5]:
# Import necessary packages.
import numpy as np
import pandas as pd
import torch
import os
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
from torchvision.datasets import DatasetFolder, VisionDataset
# This is for the progress bar.
from tqdm.auto import tqdm
import random

In [6]:
myseed = 42  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

# Transforms
Torchvision provides lots of useful utilities for image preprocessing, data *wrapping* as well as data augmentation.

Please refer to PyTorch official website for details about different transforms.

In [7]:
# Normally, We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

valid_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# However, it is also possible to use augmentation in the testing phase.
# You may use train_tfm to produce a variety of images and then test using ensemble methods
train_tfm = transforms.Compose([
    # Resize the image into a fixed shape (height = width = 128)
    transforms.Resize((128, 128)),
    # You may add some transforms here.
    transforms.RandomHorizontalFlip(p=0.3),
    transforms.RandomVerticalFlip(p=0.3),
    transforms.RandomRotation(45),
    transforms.RandomGrayscale(p=0.1),
    # ToTensor() should be the last one of the transforms.
    transforms.ToTensor(),
])

# Datasets
The data is labelled by the name, so we load images and label while calling '__getitem__'

In [8]:
class FoodDataset2(Dataset):

    def __init__(self,path1,path2,split,mode = ''):
        super(FoodDataset2).__init__()
        list1 = [os.path.join(path1,x) for x in os.listdir(path1) if x.endswith(".jpg")]
        list2 = [os.path.join(path2,x) for x in os.listdir(path2) if x.endswith(".jpg")]
        f_list = sorted(list1+list2)
        random.seed(42)
        random.shuffle(f_list)
        self.mode = mode
        self.split = split
            
        if mode == 'train':  
            list00 = []
            for i in range (10231):
                list00.append(i)
            random.shuffle(list00)
            self.reflist = list00
            
            if split == 0:
                self.files = f_list[3412:]
            elif split == 1:
                self.files = f_list[:3412]+f_list[6824:]
            elif split == 2:
                self.files = f_list[:6824]+f_list[10236:]
            elif split == 3:
                self.files = f_list[:10236]
                
            self.transform = train_tfm
        elif mode == 'valid':  

            if split == 0:
                self.files = f_list[:3412]
            elif split == 1:
                self.files = f_list[3412:6824]
            elif split == 2:
                self.files = f_list[6824:10236]
            elif split == 3:
                self.files = f_list[10236:]
                
            self.transform = valid_tfm

  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        
        if self.mode == 'train':
            if idx%4 == self.split:
                im = test_tfm(im)

                label = int(fname.split("/")[-1].split("_")[0])
                #get mixup img
                fname = self.files[self.reflist[idx%10231]]
                im_mix = Image.open(fname)
                im_mix = test_tfm(im_mix)
                label_mix = int(fname.split("/")[-1].split("_")[0])
                #do mixup
                im2 = 0.7*im + 0.3*im_mix
                label_list = torch.zeros(13)
                label_list[label] += 0.7
                label_list[label_mix] += 0.3
                label_list[11] = label
                label_list[12] = label_mix

                return im2,label_list
            else:
                im = self.transform(im)

                label = int(fname.split("/")[-1].split("_")[0])

                #Label smoothing
                label_list = torch.zeros(13)
                for i in range(11):
                    label_list[i] = 0.1
                label_list[label] = 0.9
                label_list[11] = label
                label_list[12] = label
                
                return im,label_list
        else:
            im = self.transform(im)

            label = int(fname.split("/")[-1].split("_")[0])
            
            #Label smoothing
            label_list = torch.zeros(12)
            for i in range(11):
                    label_list[i] = 0.1
            label_list[label] = 0.9
            label_list[11] = label
            
            return im,label_list

In [9]:
class FoodDataset(Dataset):

    def __init__(self,path,tfm=test_tfm,files = None, mode = ''):
        super(FoodDataset).__init__()
        self.path = path
        self.files = sorted([os.path.join(path,x) for x in os.listdir(path) if x.endswith(".jpg")])
        self.mode = mode
  
    def __len__(self):
        return len(self.files)
  
    def __getitem__(self,idx):
        fname = self.files[idx]
        im = Image.open(fname)
        
        if self.mode == 'test':
            ims = [test_tfm(im)]
            ims += [train_tfm(im) for _ in range(10)]
            label = -1 # test has no label
            return torch.stack(ims), label


# Model

In [10]:
class Residual_Block(nn.Module):
    def __init__(self, i_channel, o_channel, stride=1, down_sample=None):
        super(Residual_Block, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=i_channel, 
                    out_channels=o_channel, 
                    kernel_size=3, 
                    stride=stride, 
                    padding=1,
                    bias=False)
        self.bn1 = nn.BatchNorm2d(o_channel)
        self.relu = nn.LeakyReLU(0.1)
        self.conv2 = nn.Conv2d(in_channels=o_channel, 
                    out_channels=o_channel, 
                    kernel_size=3, 
                    stride=1, 
                    padding=1,
                    bias=False)
        self.bn2 = nn.BatchNorm2d(o_channel)
        self.down_sample = down_sample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        
        if self.down_sample:
            residual = self.down_sample(x)
        out += residual
        out = self.relu(out)

        return out

In [11]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=11):
        super(ResNet, self).__init__()
        self.conv = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1, bias=False)
        self.in_channels = 16
        self.bn = nn.BatchNorm2d(16)
        self.relu1 = nn.LeakyReLU(0.1)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[0], 2)
        self.layer3 = self.make_layer(block, 64, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc1 = nn.Linear(1024, 256)
        self.relu2 = nn.LeakyReLU(0.1)
        self.dropout = nn.Dropout(p=0.2)
        self.fc2 = nn.Linear(256, num_classes)
        
    def make_layer(self, block, out_channels, blocks, stride=1):  
        down_sample = None
        if (stride != 1) or (self.in_channels != out_channels):
            down_sample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False),
                nn.BatchNorm2d(out_channels)
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, down_sample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size()[0], -1)
        out = self.fc1(out)
        out = self.relu2(out)
        out = self.dropout(out)
        out = self.fc2(out)
        '''
        if torch.isnan(out)[0][0]:
            print('nan',x)'''
        return out

# Configurations

In [12]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)

# The number of batch size.
batch_size = 64

# The number of training epochs.
n_epochs = 210

# If no improvement in 'patience' epochs, early stop.
patience = 210

# For the classification task, we use cross-entropy as the measurement of performance.
#criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=30,T_mult=2)

# Dataloader

In [ ]:
# Construct train and valid datasets.
# The argument "loader" tells how torchvision reads the data.
k_fold_split = 2
train_set = FoodDataset2("./train","./valid",split = k_fold_split, mode='train')
valid_set = FoodDataset2("./train","./valid",split = k_fold_split, mode='valid')

print("train_set:",len(train_set.files),"valid_set:",len(valid_set.files))

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

train_set: 10231 valid_set: 3412


# Class-balanced focal loss 

In [ ]:
import torch.nn.functional as F

def focal_loss(logits, labels, alpha, gamma):
    
    bce_loss = F.binary_cross_entropy_with_logits(input=logits, target=labels, reduction="none")
    
    t0 = torch.zeros(2,logits.shape[0],11).to(device)
    t0[0] = -1.0 * logits
    
    f1 = torch. logsumexp(t0, dim=0)
    f2 = - gamma * f1
    f3 = -gamma * labels * logits

    modulator = torch.exp(f3 + f2)
    modulator = torch.nan_to_num(modulator, nan=1)
    
    loss = modulator * bce_loss
    
    weighted_loss = alpha * loss
    loss = torch.sum(weighted_loss)
    loss /= torch.sum(labels)
    
    #print((loss))
    '''
    nploss = loss.cpu().detach().numpy()
    if np.isnan(nploss):
        print(logits, labels)'''
        
    #print((loss))
    
    return loss

In [ ]:
class ClassBalancedLoss(torch.nn.Module):
    def __init__(self, samples_per_class=None, beta=0.9999, gamma=0.5):
        super(ClassBalancedLoss, self).__init__()

        effective_num = 1.0 - np.power(beta, samples_per_class)
        weights = (1.0 - beta) / np.array(effective_num)
        self.constant_sum = len(samples_per_class)
        weights = (weights / np.sum(weights) * self.constant_sum).astype(np.float32)
        self.class_weights = weights
        self.gamma = gamma


    def forward(self, x, y):

        _, num_classes = x.shape
        labels_one_hot = y.argmax(dim=-1)
        weights = torch.tensor(self.class_weights, device=x.device).index_select(0, y.argmax(dim=-1))
        weights = weights.unsqueeze(1)
        cb_loss = focal_loss(x, y, weights, self.gamma)

        return cb_loss

In [ ]:
f_len = len(train_set.files)
samples_per_class = []

for i in range(11):
    samples_per_class.append(0)

for i in range (f_len):
    fname = train_set.files[i]
    label = int(fname.split("/")[-1].split("_")[0])
    samples_per_class[label] += 1
    
print("samples_per_class:", samples_per_class)

criterion = ClassBalancedLoss(samples_per_class=samples_per_class)

samples_per_class: [1016, 1540, 448, 1371, 282, 450, 883, 1546, 717, 1063, 915]


# Start Training

In [ ]:
# Initialize trackers, these are not parameters and should not be changed
stale = 0
best_acc = 0
best_loss = 5


for epoch in range(n_epochs):

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()
        #print(imgs)

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))
        #print(logits, labels)


        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
            
        loss = criterion(logits, labels[:,:11].to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()
        
        if not torch.isnan(loss):
            # Compute the gradients for parameters.
            loss.backward()

            # Clip the gradient norms for stable training.
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

            # Update the parameters with computed gradients.
            optimizer.step()

        # Compute the accuracy for current batch.
        label_1_acc = (logits.argmax(dim=-1) == labels[:,11].to(device)).float().mean()
        label_2_acc = (logits.argmax(dim=-1) == labels[:,12].to(device)).float().mean()
        acc = 0.7*label_1_acc + 0.3*label_2_acc

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)
        
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)
    
    
    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    scheduler.step()
    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch
        #imgs = imgs.half()

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
            logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels[:,:11].to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels[:,11].to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)
        #break

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    #print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # update logs
    if valid_acc > best_acc:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best acc")
    else:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")


    # save models
    if valid_acc > best_acc:
        print(f"Best acc model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best.ckpt") # only save best to prevent output memory exceed error
        best_acc = valid_acc
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f"No improvment {patience} consecutive epochs, early stopping")
            break
    
    #for label smoothing
    if valid_loss < best_loss:
        with open(f"./{_exp_name}_log.txt","a"):
            print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f} -> best loss")
        print(f"Best loss model found at epoch {epoch}, saving model")
        torch.save(model.state_dict(), f"{_exp_name}_best_loss.ckpt") # only save best to prevent output memory exceed error
        best_loss = valid_loss

  0%|          | 0/80 [00:00<?, ?it/s]

[ Train | 001/250 ] loss = 1.71099, acc = 0.17693


  0%|          | 0/27 [00:00<?, ?it/s]

[ Valid | 001/250 ] loss = 1.60942, acc = 0.26226 -> best acc
Best acc model found at epoch 0, saving model
[ Valid | 001/250 ] loss = 1.60942, acc = 0.26226 -> best loss
Best loss model found at epoch 0, saving model


  0%|          | 0/80 [00:00<?, ?it/s]

...

  0%|          | 0/80 [00:00<?, ?it/s]

[ Train | 050/250 ] loss = 0.68727, acc = 0.67475


  0%|          | 0/27 [00:00<?, ?it/s]

[ Valid | 050/250 ] loss = 1.18930, acc = 0.61890


  0%|          | 0/80 [00:00<?, ?it/s]

# Dataloader for test

In [13]:
# Construct test datasets.
# The argument "loader" tells how torchvision reads the data.
test_set = FoodDataset("./test", tfm=test_tfm, mode = 'test')
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

# Testing and generate prediction CSV

In [14]:
model_best = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best1 = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best2 = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best3 = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best.load_state_dict(torch.load(f"/content/0316kf0_best.ckpt"))
model_best1.load_state_dict(torch.load(f"/content/0315kf1_best.ckpt"))
model_best2.load_state_dict(torch.load(f"/content/0317kf2_best.ckpt"))
model_best3.load_state_dict(torch.load(f"/content/0317kf3_best.ckpt"))
model_best.eval()
model_best1.eval()
model_best2.eval()
model_best3.eval()
test_pred1 = []
test_pred2 = []
test_pred3 = []
test_pred4 = []
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred1 = []
        # TTA
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred1.append(logit)
            
        test_pred1 = torch.stack(test_pred1).cpu().data.numpy()

        test_pred2 = []
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best1(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best1(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred2.append(logit)
            
        test_pred2 = torch.stack(test_pred2).cpu().data.numpy()

        test_pred3 = []
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best2(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best2(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred3.append(logit)
            
        test_pred3 = torch.stack(test_pred3).cpu().data.numpy()

        test_pred4 = []
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best3(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best3(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred4.append(logit)
            
        test_pred4 = torch.stack(test_pred4).cpu().data.numpy()
        
        test_label = np.argmax(test_pred1+test_pred2+test_pred3+test_pred4, axis=1)
        prediction += test_label.squeeze().tolist()  
        


  0%|          | 0/47 [00:00<?, ?it/s]

In [24]:
model_best = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best1 = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best.load_state_dict(torch.load(f"/content/0317kf2_best.ckpt"))
model_best1.load_state_dict(torch.load(f"/content/0317kf3_best.ckpt"))
model_best.eval()
model_best1.eval()
test_pred1 = []
test_pred2 = []
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        test_pred1 = []
        # TTA
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred1.append(logit)
            
        test_pred1 = torch.stack(test_pred1).cpu().data.numpy()

        test_pred2 = []
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best1(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best1(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred2.append(logit)
            
        test_pred2 = torch.stack(test_pred2).cpu().data.numpy()


        
        test_label = np.argmax(test_pred1+test_pred2, axis=1)
        prediction += test_label.squeeze().tolist()  
        


  0%|          | 0/47 [00:00<?, ?it/s]

In [25]:
# create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission.csv",index = False)

In [26]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Label smoothing

In [21]:
model_best = ResNet(Residual_Block, [2, 2, 2, 2]).to(device)
model_best.load_state_dict(torch.load(f"/content/0317kf3_best.ckpt"))
model_best.eval()
prediction = []
with torch.no_grad():
    for data,_ in tqdm(test_loader):
        
        # TTA
        test_pred = []
        for imgs in data:
            imgs_first = imgs[0].unsqueeze(0)
            origin_logit = model_best(imgs_first.to(device)).squeeze(0)
            tta_logit = model_best(imgs[1:].to(device))
            tta_logit = torch.mean(tta_logit, 0)
            logit = (0.6*origin_logit) + (0.4*tta_logit)
            test_pred.append(logit)
            
        test_pred = torch.stack(test_pred)
        
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        prediction += test_label.squeeze().tolist()  

  0%|          | 0/47 [00:00<?, ?it/s]

In [22]:
#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission_LB.csv",index = False)

In [23]:
from google.colab import files
files.download('submission_LB.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Voting

In [51]:
from collections import Counter
import csv

with open(f"/content/submission_0317kf2.csv", newline='') as f:
  reader = csv.reader(f)
  content1 = []
  for row in reader:
    content1.append(row[1])
  content1 = content1[1:]
with open(f"/content/submission_0317kf3.csv", newline='') as f:
  reader = csv.reader(f)
  content2 = []
  for row in reader:
    content2.append(row[1])
  content2 = content2[1:]
with open(f"/content/submission.csv", newline='') as f:
  reader = csv.reader(f)
  content3 = []
  for row in reader:
    content3.append(row[1])
  content3 = content3[1:]
with open(f"/content/0313v1.csv", newline='') as f:
  reader = csv.reader(f)
  content4 = []
  for row in reader:
    content4.append(row[1])
  content4 = content4[1:]

prediction = []
for i in range(len(test_set)):
  x = Counter([content4[i], content1[i], content2[i], content3[i]])
  prediction.append(x.most_common(1)[0][0])
print(prediction, len(prediction))

#create test csv
def pad4(i):
    return "0"*(4-len(str(i)))+str(i)
df = pd.DataFrame()
df["Id"] = [pad4(i) for i in range(len(test_set))]
df["Category"] = prediction
df.to_csv("submission_vote.csv",index = False)

['5', '1', '0', '3', '9', '10', '3', '6', '1', '3', '3', '1', '8', '6', '3', '3', '3', '3', '9', '9', '2', '9', '4', '6', '2', '1', '4', '2', '3', '6', '1', '4', '0', '1', '8', '10', '7', '10', '10', '7', '4', '1', '1', '8', '9', '6', '3', '10', '1', '0', '3', '6', '2', '10', '10', '10', '5', '10', '6', '10', '4', '4', '1', '8', '2', '3', '7', '3', '8', '1', '1', '9', '6', '10', '7', '1', '3', '2', '1', '8', '9', '6', '3', '10', '0', '10', '0', '1', '1', '9', '10', '7', '9', '9', '7', '7', '6', '7', '4', '5', '5', '0', '3', '10', '9', '7', '7', '2', '5', '1', '3', '1', '9', '1', '1', '6', '3', '8', '0', '7', '8', '9', '6', '2', '3', '7', '10', '3', '1', '1', '1', '8', '6', '7', '3', '3', '10', '1', '2', '10', '8', '9', '10', '0', '7', '3', '3', '9', '10', '9', '3', '3', '10', '3', '7', '7', '3', '0', '1', '0', '3', '0', '3', '6', '1', '3', '8', '2', '3', '9', '1', '1', '7', '1', '8', '1', '1', '0', '3', '5', '1', '10', '7', '6', '1', '2', '10', '0', '9', '9', '7', '10', '10', '5', '8',